In [9]:
from datasets import load_dataset

dataset = load_dataset("kqsong/OASum", split = "test", cache_dir='/scratch/ramprasad.sa/huggingface_datasets')

In [10]:
dataset[0].keys()

dict_keys(['title', 'document', 'aspect', 'aspect_sents', 'summary'])

In [16]:
import pandas as pd
import random 
from tqdm import tqdm 
def make_oasumm_data():
    dataset = load_dataset("kqsong/OASum", split = "test", cache_dir='/scratch/ramprasad.sa/huggingface_datasets')
    df_processed = {
        'id': [],
        'Title': [],
        'Article_OG': [],
        'Article': [],
        'Aspect': [],
        'Aspect_Sents': [],
        'Query': [],
        'Reference': [],
        'Faithful_Query': [],
        
    }
    for datap in tqdm(dataset):
        id = random.getrandbits(128)
        title = datap['title']
        doc = datap['document']
        asp_sents_idx = datap['aspect_sents']
        asp = datap['aspect']
        summ = datap['summary']
        
        doc_wo_aspect_sents = [each for idx, each in enumerate(doc) if idx not in asp_sents_idx]
        asp_sents = [doc[idx] for idx in asp_sents_idx]
        aspect_query = f'Summarize the {asp} of {title}'
        
        df_processed['id'] += [id]
        df_processed['Title'] += [title]
        df_processed['Article_OG'] += [' '.join(doc)]
        df_processed['Article'] += [' '.join(doc_wo_aspect_sents)]
        df_processed['Aspect'] += [asp]
        df_processed['Aspect_Sents'] += [' '.join(asp_sents)]
        df_processed['Query'] += [aspect_query]
        df_processed['Reference'] += [' '.join(summ)]
        df_processed['Faithful_Query']+= ['no']
        
    return pd.DataFrame(df_processed)
        

In [17]:
df_oasumm = make_oasumm_data()



100%|██████████| 112005/112005 [00:34<00:00, 3222.65it/s]


In [18]:
df_oasumm.head()

,id,Title,Article_OG,Article,Aspect,Aspect_Sents,Query,Reference,Faithful_Query
0,24256940101438506901381621671727034209,Cauchy elastic material,"Formally, a material is said to be Cauchy-elas...",For an isotropic material the Cauchy stress te...,Mathematical definition,"Formally, a material is said to be Cauchy-elas...",Summarize the Mathematical definition of Cauch...,"Finally, a Cauchy-elastic material must satisf...",no
1,208739999386664684703659817589895092784,Cauchy elastic material,"Formally, a material is said to be Cauchy-elas...","Formally, a material is said to be Cauchy-elas...",Non-conservative materials,Even though the stress in a Cauchy-elastic mat...,Summarize the Non-conservative materials of Ca...,A Cauchy-elastic material is also called a sim...,no
2,46697146211713115688501056885404503140,Comic LO,"When first published, Comic LO helped trigger ...",,History,"When first published, Comic LO helped trigger ...",Summarize the History of Comic LO,"is a Japanese ""lolicon""-focused erotic manga m...",no
3,76983386080562055217023388263327970432,Druid of Colchester,The grave was discovered by archaeologists in ...,The grave was discovered by archaeologists in ...,Artifacts,"In the wooden chambered burial site, archaeolo...",Summarize the Artifacts of Druid of Colchester,The person whose cremated remains were buried ...,no
4,122421538055764622262442344867864865497,Maeve Fort,"Born on 19 November 1940 in Liverpool, Fort wa...","Born on 19 November 1940 in Liverpool, Fort wa...",Foreign Office,Fort decided to apply to join the Foreign Offi...,Summarize the Foreign Office of Maeve Fort,During her posting as the United Kingdom's Hig...,no


In [19]:
df_oasumm.to_csv('/home/ramprasad.sa/qf_sd_summarization/datasets/postprocessed/aspect_focused/oasumm.csv')